# V2 Sign Optimization Validation

This notebook validates the V2 layer-by-layer energy minimization approach for resolving nematic sign ambiguity.

**Key Concept**: Instead of BFS from a seed point, V2:
1. Propagates signs layer-by-layer (z=0 → z=max)
2. For each layer, aligns with z-1 to minimize gradient energy
3. Refines with global iterative passes

In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import fcpm
from fcpm.reconstruction import reconstruct_via_qtensor, combined_optimization
from sign_optimization_v2 import (
    layer_then_refine,
    layer_by_layer_vectorized,
    compute_gradient_energy
)
import time

print(f"FCPM version: {fcpm.__version__}")

FCPM version: 1.1.0


## Setup Test Data

In [2]:
# Create cholesteric director (helical structure - natural for z-layer optimization)
shape = (48, 48, 24)
director_gt = fcpm.create_cholesteric_director(shape=shape, pitch=6.0)

# Simulate FCPM with noise
noise_level = 0.05  # 5% noise
I_fcpm = fcpm.simulate_fcpm(director_gt)
I_fcpm_noisy = fcpm.add_fcpm_realistic_noise(I_fcpm, noise_model='mixed', gaussian_sigma=noise_level)
I_fcpm_noisy = fcpm.normalize_fcpm(I_fcpm_noisy)

print(f"Director shape: {director_gt.shape}")
print(f"Noise level: {noise_level*100:.0f}%")

Director shape: (48, 48, 24)
Noise level: 5%


## Q-Tensor Reconstruction (No Sign Fix)

In [3]:
# Reconstruct without sign optimization
director_raw, Q, info = reconstruct_via_qtensor(I_fcpm_noisy)

raw_energy = compute_gradient_energy(director_raw.to_array())
raw_metrics = fcpm.summary_metrics(director_raw, director_gt)

print(f"Raw reconstruction (no sign fix):")
print(f"  Gradient energy: {raw_energy:.2f}")
print(f"  Angular error: {raw_metrics['angular_error_mean_deg']:.2f} deg (mean)")

Raw reconstruction (no sign fix):
  Gradient energy: 132065.46
  Angular error: 7.38 deg (mean)


## V1 Optimization (BFS + Iterative Flip)

In [4]:
t0 = time.time()
director_v1, info_v1 = combined_optimization(director_raw, verbose=True)
time_v1 = time.time() - t0

v1_metrics = fcpm.summary_metrics(director_v1, director_gt)
print(f"\nV1 Results:")
print(f"  Time: {time_v1:.3f}s")
print(f"  Final energy: {info_v1['final_energy']:.2f}")
print(f"  Angular error: {v1_metrics['angular_error_mean_deg']:.2f} deg")

Combined Sign Optimization

Phase 1: Chain propagation...
  After chain propagation: energy = 60687.96

Phase 2: Local flip refinement...
Initial gradient energy: 60687.96
  Iter 0: flipped 0 voxels, energy = 60687.96
Converged at iteration 0

V1 Results:
  Time: 0.083s
  Final energy: 60687.96
  Angular error: 7.38 deg


## V2 Optimization (Layer-by-Layer + Refinement)

In [5]:
t0 = time.time()
result_v2 = layer_then_refine(director_raw, verbose=True)
time_v2 = time.time() - t0

v2_metrics = fcpm.summary_metrics(result_v2.director, director_gt)
print(f"\nV2 Results:")
print(f"  Time: {time_v2:.3f}s")
print(f"  Final energy: {result_v2.final_energy:.2f}")
print(f"  Angular error: {v2_metrics['angular_error_mean_deg']:.2f} deg")

V2 Layer-Then-Refine Optimization
Initial energy: 132065.46

Phase 1: Layer propagation (z-1 reference)...
  After layer propagation: energy = 60687.96

Phase 2: Iterative refinement...
  Iter 0: flipped 0 voxels, energy = 60687.96
  Converged at iteration 0

Final energy: 60687.96
Total reduction: 71377.50 (54.0%)

V2 Results:
  Time: 0.017s
  Final energy: 60687.96
  Angular error: 7.38 deg


## Comparison Summary

In [6]:
print("="*60)
print("COMPARISON SUMMARY")
print("="*60)
print(f"{'Method':<25} {'Energy':>12} {'Ang. Error':>12} {'Time':>10}")
print("-"*60)
print(f"{'Raw (no fix)':<25} {raw_energy:>12.2f} {raw_metrics['angular_error_mean_deg']:>10.2f}° {'-':>10}")
print(f"{'V1 (BFS + flip)':<25} {info_v1['final_energy']:>12.2f} {v1_metrics['angular_error_mean_deg']:>10.2f}° {time_v1:>9.3f}s")
print(f"{'V2 (layer + refine)':<25} {result_v2.final_energy:>12.2f} {v2_metrics['angular_error_mean_deg']:>10.2f}° {time_v2:>9.3f}s")
print("-"*60)

speedup = time_v1 / time_v2 if time_v2 > 0 else float('inf')
print(f"\nV2 Speedup: {speedup:.1f}x")

energy_match = abs(info_v1['final_energy'] - result_v2.final_energy) < 100
print(f"Energy match: {'Yes' if energy_match else 'No'}")

COMPARISON SUMMARY
Method                          Energy   Ang. Error       Time
------------------------------------------------------------
Raw (no fix)                 132065.46       7.38°          -
V1 (BFS + flip)               60687.96       7.38°     0.083s
V2 (layer + refine)           60687.96       7.38°     0.017s
------------------------------------------------------------

V2 Speedup: 4.8x
Energy match: Yes


## Visual Comparison

In [7]:
z_mid = director_gt.shape[2] // 2

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Row 1: Director slices
fcpm.plot_director_slice(director_gt, z_idx=z_mid, step=2, ax=axes[0, 0], title='Ground Truth')
fcpm.plot_director_slice(director_raw, z_idx=z_mid, step=2, ax=axes[0, 1], title='Raw (no fix)')
fcpm.plot_director_slice(director_v1, z_idx=z_mid, step=2, ax=axes[0, 2], title='V1 Optimized')
fcpm.plot_director_slice(result_v2.director, z_idx=z_mid, step=2, ax=axes[0, 3], title='V2 Optimized')

# Row 2: nz component (often shows sign issues)
im0 = axes[1, 0].imshow(director_gt.nz[:, :, z_mid], cmap='RdBu_r', vmin=-1, vmax=1)
axes[1, 0].set_title('GT nz')
axes[1, 0].axis('off')

im1 = axes[1, 1].imshow(director_raw.nz[:, :, z_mid], cmap='RdBu_r', vmin=-1, vmax=1)
axes[1, 1].set_title('Raw nz')
axes[1, 1].axis('off')

im2 = axes[1, 2].imshow(director_v1.nz[:, :, z_mid], cmap='RdBu_r', vmin=-1, vmax=1)
axes[1, 2].set_title('V1 nz')
axes[1, 2].axis('off')

im3 = axes[1, 3].imshow(result_v2.director.nz[:, :, z_mid], cmap='RdBu_r', vmin=-1, vmax=1)
axes[1, 3].set_title('V2 nz')
axes[1, 3].axis('off')

plt.suptitle(f'V1 vs V2 Comparison (z={z_mid}, noise={noise_level*100:.0f}%)', fontsize=14)
plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_6460/2704422209.py:30: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Test with Different Noise Levels

In [8]:
noise_levels = [0.01, 0.03, 0.05, 0.08, 0.10]
v1_energies = []
v2_energies = []
v1_times = []
v2_times = []

for noise in noise_levels:
    # Simulate
    I_noisy = fcpm.add_fcpm_realistic_noise(I_fcpm, noise_model='mixed', gaussian_sigma=noise)
    I_noisy = fcpm.normalize_fcpm(I_noisy)
    d_raw, _, _ = reconstruct_via_qtensor(I_noisy)
    
    # V1
    t0 = time.time()
    d_v1, info_v1 = combined_optimization(d_raw, verbose=False)
    v1_times.append(time.time() - t0)
    v1_energies.append(info_v1['final_energy'])
    
    # V2
    t0 = time.time()
    result_v2 = layer_then_refine(d_raw, verbose=False)
    v2_times.append(time.time() - t0)
    v2_energies.append(result_v2.final_energy)
    
    print(f"Noise {noise*100:>4.0f}%: V1={info_v1['final_energy']:.0f}, V2={result_v2.final_energy:.0f}")

Noise    1%: V1=56622, V2=56622
Noise    3%: V1=58041, V2=58041
Noise    5%: V1=60655, V2=60655
Noise    8%: V1=64790, V2=64790
Noise   10%: V1=67338, V2=67338


In [9]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

noise_pct = [n*100 for n in noise_levels]

axes[0].plot(noise_pct, v1_energies, 'b-o', label='V1', linewidth=2, markersize=8)
axes[0].plot(noise_pct, v2_energies, 'r-s', label='V2', linewidth=2, markersize=8)
axes[0].set_xlabel('Noise Level (%)', fontsize=12)
axes[0].set_ylabel('Final Energy', fontsize=12)
axes[0].set_title('Energy vs Noise', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(noise_pct, v1_times, 'b-o', label='V1', linewidth=2, markersize=8)
axes[1].plot(noise_pct, v2_times, 'r-s', label='V2', linewidth=2, markersize=8)
axes[1].set_xlabel('Noise Level (%)', fontsize=12)
axes[1].set_ylabel('Time (seconds)', fontsize=12)
axes[1].set_title('Computation Time', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

/var/folders/49/56t9h5ws6s31gccfg9d7kn9m0000gn/T/ipykernel_6460/1609253921.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Conclusion

V2 `layer_then_refine` achieves:
- **Same energy** as V1 at moderate noise levels (≤10%)
- **4x speedup** on average
- **Cleaner conceptual framework** (layer-by-layer mirrors z-stack acquisition)

For very high noise (>10%), V1's global BFS may still find better solutions.